In [18]:
from tabula import read_pdf
import pandas as pd
import os


# Extraer tablas de un PDF1
tablas = read_pdf("../Limpieza_Accidentes_Transito/pdf/RED VIAL VECINAL.pdf", pages="all", multiple_tables=True, encoding='utf-8')

In [19]:
df = pd.DataFrame(tablas[3])
df.head()

,Unnamed: 0,ID,Unnamed: 1,Unnamed: 2,ÓDIG,Unnamed: 3,Unnamed: 4,TRAYECTORIA,Unnamed: 5
0,7,AM-506,Emp. PE-5N C - Imacita - Puerto Imacita.,NaN,NaN,NaN,NaN,NaN,NaN
1,8,AM-507,Emp. PE-5N C - Tec Sec - Umukai - Pta.Carretera.,NaN,NaN,NaN,NaN,NaN,NaN
2,9,AM-508,Emp. PE-5N C - Pta. Carretera.,NaN,NaN,NaN,NaN,NaN,NaN
3,10,AM-509,Emp. PE-5N C - Inayo - Puerto Pakuy.,NaN,NaN,NaN,NaN,NaN,NaN
4,11,AM-510,Emp. PE-5N C - Aramango - Pte. Cotrina - Pte. ...,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df.columns

Index(['Unnamed: 0', 'ID', 'Unnamed: 1', 'Unnamed: 2', 'ÓDIG', 'Unnamed: 3',
       'Unnamed: 4', 'TRAYECTORIA', 'Unnamed: 5'],
      dtype='object')

In [21]:
df.columns = ['COLUMN1', 'ID', 'TRAYECTORIA', 'COLUMN4', 'COLUMN5', 'COLUMN6', 'COLUMN7', 'COLUMN8', 'COLUMN9']
df.head()


,COLUMN1,ID,TRAYECTORIA,COLUMN4,COLUMN5,COLUMN6,COLUMN7,COLUMN8,COLUMN9
0,7,AM-506,Emp. PE-5N C - Imacita - Puerto Imacita.,NaN,NaN,NaN,NaN,NaN,NaN
1,8,AM-507,Emp. PE-5N C - Tec Sec - Umukai - Pta.Carretera.,NaN,NaN,NaN,NaN,NaN,NaN
2,9,AM-508,Emp. PE-5N C - Pta. Carretera.,NaN,NaN,NaN,NaN,NaN,NaN
3,10,AM-509,Emp. PE-5N C - Inayo - Puerto Pakuy.,NaN,NaN,NaN,NaN,NaN,NaN
4,11,AM-510,Emp. PE-5N C - Aramango - Pte. Cotrina - Pte. ...,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
columns_to_drop = ['COLUMN1', 'COLUMN4', 'COLUMN5', 'COLUMN6', 'COLUMN7', 'COLUMN8', 'COLUMN9']
for column in columns_to_drop:
    df.drop(column, axis=1, inplace=True)


In [23]:
df.head()

,ID,TRAYECTORIA
0,AM-506,Emp. PE-5N C - Imacita - Puerto Imacita.
1,AM-507,Emp. PE-5N C - Tec Sec - Umukai - Pta.Carretera.
2,AM-508,Emp. PE-5N C - Pta. Carretera.
3,AM-509,Emp. PE-5N C - Inayo - Puerto Pakuy.
4,AM-510,Emp. PE-5N C - Aramango - Pte. Cotrina - Pte. ...


In [24]:
cantidad_tablas = len(tablas)
print(f"El PDF contiene {cantidad_tablas} tablas.")

El PDF contiene 323 tablas.


In [25]:


csv_file_path = "../Limpieza_Accidentes_Transito/red_vial_vecinal.xlsx"

# Crear la carpeta si no existe
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)

# Eliminar el archivo si ya existe
if os.path.exists(csv_file_path):
    os.remove(csv_file_path)

tablas_procesadas = []  # Para guardar las tablas que cumplen el formato
tablas_no_conformes = []  # Para guardar información sobre tablas que no cumplen

# Definir encabezados esperados (primeros encabezados)
encabezados_esperados = ['ID', 'CÓDIGO', 'Trayectoria']

for i, tabla in enumerate(tablas[2:], start=2):
    print(f"Procesando Tabla {i + 1}...")
    df = pd.DataFrame(tabla)  # Convertir la tabla a DataFrame

    # Verificar si la tabla tiene al menos 3 columnas
    if df.shape[1] < 3:
        print(f"Tabla {i + 1} ignorada (menos de 3 columnas).")
        tablas_no_conformes.append({"indice": i + 1, "razon": "menos de 3 columnas", "contenido": df})
        continue

    # Seleccionar solo las columnas 0, 1 y 2
    df = df.iloc[:, [0, 1, 2]]

    # Verificar si la tabla está vacía o sin datos válidos
    if df.isnull().all(axis=1).sum() == df.shape[0]:
        print(f"Tabla {i + 1} ignorada (todas las filas vacías).")
        tablas_no_conformes.append({"indice": i + 1, "razon": "todas las filas vacías", "contenido": df})
        continue

    # Renombrar columnas solo para la primera tabla
    if len(tablas_procesadas) == 0:  # Si es la primera tabla
        df.columns = encabezados_esperados
    else:  # Para las demás tablas, mantener columnas uniformes
        df.columns = encabezados_esperados

    # Convertir la columna 'ID' a tipo string para evitar errores
    df['ID'] = df['ID'].astype(str)

    # Eliminar posibles encabezados repetidos en las demás tablas
    df = df[~df['ID'].str.contains('ID|CÓDIGO|TRAYECTORIA', na=False)]

    # Agregar tabla procesada a la lista
    tablas_procesadas.append(df)

    # Mostrar la tabla procesada
    print(df)
    print("\n" + "-" * 50 + "\n")

# Mostrar las tablas que no cumplieron el formato
print("\nTablas No Conformes:")
for item in tablas_no_conformes:
    print(f"Tabla {item['indice']}: {item['razon']}")
    print(item['contenido'])
    print("\n" + "-" * 50 + "\n")

# Combinar todas las tablas procesadas
if len(tablas_procesadas) > 0:
    # Concatenar todas las tablas
    df_final = pd.concat(tablas_procesadas, ignore_index=True)
    
    # Exportar a Excel
    df_final.to_excel("../Limpieza_Accidentes_Transito/excel/Red_vial_vecinal_Limpio.xlsx", index=False, engine='openpyxl')
    print(f"Archivo exportado exitosamente a {csv_file_path}")
else:
    print("No se procesaron tablas conformes.")



Procesando Tabla 3...
  ID  CÓDIGO                    Trayectoria
0  1  AM-500          Emp. PE-5N D – Hebrón
1  2  AM-501       Emp. PE-5N D – Chingamar
2  3  AM-502  Emp. PE-5N D - Pta.Carretera.
3  4  AM-503  Emp. PE-5N D - Pta.Carretera.
4  5  AM-504        Emp. PE-5N D – Chiangos
5  6  AM-505       Emp. PE-5N C – Kusumatak

--------------------------------------------------

Procesando Tabla 4...
    ID  CÓDIGO                                        Trayectoria
0    7  AM-506           Emp. PE-5N C - Imacita - Puerto Imacita.
1    8  AM-507   Emp. PE-5N C - Tec Sec - Umukai - Pta.Carretera.
2    9  AM-508                     Emp. PE-5N C - Pta. Carretera.
3   10  AM-509               Emp. PE-5N C - Inayo - Puerto Pakuy.
4   11  AM-510  Emp. PE-5N C - Aramango - Pte. Cotrina - Pte. ...
5   12  AM-511           Emp. AM-510 (Aramango) - Pta. Carretera.
6   13  AM-512  Emp. PE-5N C - Monte Seco - El Porvenir - Pta....
7   14  AM-513  Emp. PE-5N C - Valencia - Guayaquil - Pte. Nic...
8